In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('Sales.csv')

In [8]:
print("Original Dataset:")
display(df)
print("\n\n")


Original Dataset:


,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,01/02/2024,Widget B,5.0,40.0,200.0
2,1003,NaN,2024/01/03',Widget A,NaN,25.0,NaN
3,1004,Alice Johnson,04/01/2024,Widget C,3.0,NaN,210.0
4,1005,Bob Brown,2024/01/05',Widget B,10.0,40.0,400.0
5,1006,John Doe,06/01/2024,Widget A,4.0,25.0,100.0
6,1001,John Doe,01/01/2024,Widget A,10.0,25.0,250.0
7,1007,Jane Smith,07/01/2024,Widget C,-6.0,70.0,-420.0


In [9]:
df['Customer Name'] = df['Customer Name'].fillna('Unknown')

In [10]:
df['Quantity'] = df['Quantity'].fillna(0)
df['Total Revenue'] = df['Total Revenue'].fillna(0)

In [11]:
for index, row in df.iterrows():
    if pd.isna(row['Unit Price']):
        product = row['Product']
        # Try to calculate from Total Revenue and Quantity if possible
        if row['Total Revenue'] != 0 and row['Quantity'] != 0:
            df.at[index, 'Unit Price'] = row['Total Revenue'] / row['Quantity']
        else:
            # Use median price for that product
            median_price = df[df['Product'] == product]['Unit Price'].median()
            df.at[index, 'Unit Price'] = median_price

In [12]:
df['Order Date'] = df['Order Date'].str.replace("'", "")

# Then convert to datetime, trying multiple formats
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce', format='mixed')

In [13]:
df = df.dropna(subset=['Order Date'])

# 3. Remove Duplicate Rows
df = df.drop_duplicates()

In [14]:
df['Quantity'] = df['Quantity'].abs()
df['Total Revenue'] = df['Total Revenue'].abs()

In [15]:
for index, row in df.iterrows():
    calculated_revenue = row['Quantity'] * row['Unit Price']
    # If the existing revenue is more than 10% different from calculated, update it
    if abs(row['Total Revenue'] - calculated_revenue) > (0.1 * calculated_revenue):
        df.at[index, 'Total Revenue'] = calculated_revenue

In [16]:
print("Cleaned Dataset:")
display(df)

# Save the cleaned dataset to a new CSV file
df.to_csv('Cleaned_Sales.csv', index=False)
print("\nCleaned dataset saved to 'Cleaned_Sales.csv'")

Cleaned Dataset:


,Order ID,Customer Name,Order Date,Product,Quantity,Unit Price,Total Revenue
0,1001,John Doe,2024-01-01,Widget A,10.0,25.0,250.0
1,1002,Jane Smith,2024-01-02,Widget B,5.0,40.0,200.0
2,1003,Unknown,2024-01-03,Widget A,0.0,25.0,0.0
3,1004,Alice Johnson,2024-04-01,Widget C,3.0,70.0,210.0
4,1005,Bob Brown,2024-01-05,Widget B,10.0,40.0,400.0
5,1006,John Doe,2024-06-01,Widget A,4.0,25.0,100.0
7,1007,Jane Smith,2024-07-01,Widget C,6.0,70.0,420.0



Cleaned dataset saved to 'Cleaned_Sales.csv'
